# LSTM(长短时记忆网络)
LSTM 是一种特殊的循环神经网络（RNN），能够有效地处理和记忆长期依赖关系。它的核心优势是解决了标准 RNN 在处理长序列时的梯度消失和梯度爆炸问题。
<center><img src="https://i-blog.csdnimg.cn/blog_migrate/0ec26f5e7ee914ea98d6a4ee0f2d8014.png"></center>

**LSTM 由以下两个主要部分构成：**

**Cell state (细胞状态)：**这是 LSTM 最核心的部分，充当了网络的“记忆”角色。它可以在时间步长之间传递信息，且信息可以被修改或者保留。

三个门：它们控制着信息的流动，包括：

- 遗忘门 (Forget Gate)：决定哪些信息从细胞状态中“丢弃”。

- 输入门 (Input Gate)：决定哪些新信息需要被添加到细胞状态中。

- 输出门 (Output Gate)：决定从细胞状态中提取哪些信息作为输出。

## (1)LSTM的工作流程

### 1.遗忘门(Forget Gate)

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/2e697db05c0f6ba33a33b6933f8b1a18.png"></center>

输入：当前时间步的输入 x_t 和上一时间步的隐藏状态 h_{t-1}

输出：一个介于 0 和 1 之间的数值（使用 Sigmoid 函数），表示每一部分细胞状态应该被“遗忘”的比例。

>Sigmoid 激活函数与 tanh 函数类似，不同之处在于 sigmoid 是把值压缩到0~1 之间而不是 -1~1 之间。这样的设置有助于更新或忘记信息：
- 因为任何数乘以 0 都得 0，这部分信息就会剔除掉；
- 同样的，任何数乘以 1 都得到它本身，这部分信息就会完美地保存下来
>相当于要么是1则记住，要么是0则忘掉，所以还是这个原则：因记忆能力有限，记住重要的，忘记无关紧要的。

数学表达式：
$$f_{t}=\sigma\left(W_{f} \cdot\left[h_{t-1}, x_{t}\right]+b_{f}\right)$$

### 2.输入门(Input Gate)

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/399749776a03b6551922b9d1738e3dfe.png"></center>

输入：当前时间步的输入 x_t 和上一时间步的隐藏状态 h_{t-1}

输出：

1. 更新内容：决定新的候选状态$\tilde{C}_{t}$, 即当前时间步可以储存的信息

2. 更新量：通过 Sigmoid 激活的输入门控制着有多少新信息进入细胞状态

数学表达式：
$$\begin{array}{c}
i_{t}=\sigma\left(W_{i} \cdot\left[h_{t-1}, x_{t}\right]+b_{i}\right) \\
\tilde{C}_{t}=\tanh \left(W_{C} \cdot\left[h_{t-1}, x_{t}\right]+b_{C}\right)
\end{array}$$

其中i_t是输入们的输出

## 3.细胞状态更新(Cell State Update)

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/399749776a03b6551922b9d1738e3dfe.png"></center>

根据遗忘门和输入门的输出，更新细胞状态。具体来说：

>细胞状态是上一时刻的细胞状态C_{t-1}经过遗忘门的输出缩放（遗忘掉某些信息），然后加上输入门的输出更新信息

数学表达式：

$$C_{t}=f_{t} * C_{t-1}+i_{t} * \tilde{C}_{t}$$

其中C_{t}为当前时刻的细胞状态

## 4.输出门(Output Gate)

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/8289b41615409092666db6ffdf594ffc.png"></center>

输出门决定了当前时刻的隐藏状态h_t，即输出值

数学表达式：

$$\begin{array}{c}
o_{t}=\sigma\left(W_{o} \cdot\left[h_{t-1}, x_{t}\right]+b_{o}\right) \\
h_{t}=o_{t} * \tanh \left(C_{t}\right)
\end{array}$$

## 5.总结

LSTM的核心思想：

遗忘门：决定保留多少过去的信息。

输入门：决定当前输入的信息有多少需要添加到“记忆”中。

输出门：决定从“记忆”中提取多少信息作为当前的输出。

LSTM通过这些门控机制，实现了有效的记忆和信息过滤，从而解决了标准 RNN 长期依赖问题。

## 6.代码任务

实现一个简单的 LSTM Cell（只包括核心计算部分），代码如下：

In [ ]:
import numpy as np

class LSTMCell:
    def __init__(self, input_size, hidden_size):
        # 权重初始化
        self.W_f = np.random.randn(hidden_size, input_size + hidden_size) #遗忘门权重
        self.b_f = np.zeros(hidden_size, 1)

        self.W_i = np.random.randn(hidden_size, input_size + hidden_size) #输入门权重
        self.b_i = np.zeros(hidden_size, 1)

        self.W_C = np.random.randn(hidden_size, input_size + hidden_size) #候选状态权重
        self.b_C = np.zeros(hidden_size, 1)

        self.W_o = np.random.randn(hidden_size, input_size + hidden_size) #输出门权重
        self.b_o = np.zeros(hidden_size, 1)

    def forward(self, x_t, h_prev, C_prev):
        """
        x_t: 当前时间步的输入
        h_prev: 上一时间步的隐藏状态
        C_prev: 上一时间步的细胞状态
        """
        # 拼接输入和上一时间步的隐藏状态
        combined = np.concatenate(x_t, h_prev)

        # forget gate
        f_t = self.sigmoid(np.dot(self.W_f, combined) + self.b_f)

        # input gate
        i_t = self.sigmoid(np.dot(self.W_i, combined) + self.b_i)

        #  calculate candinate state
        C_tilde = np.tanh(np.dot(self.W_C, combined) + self.b_C)

        # update cell state
        C_t = f_t * h_prev + i_t * C_tilde

        # output gate
        o_t = self.sigmoid(np.dot(self.W_o, combined) + self.b_o)

        # 计算隐藏状态
        h_t = o_t * np.tanh(C_t)

        return h_t, C_t

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

## LSTM的变体GRU

GatedRecurrentUnit(GRU)，这是由Cho,etal.(2014)提出。

它将忘记门和输入门合成了一个单一的更新门。同时还混合了细胞状态和隐藏状态。最终的模型比标准的LSTM模型要简单，是非常流行的变体。

|              | LSTM                       | GRU               |
| ------------ | -------------------------- | ----------------- |
| 门的数量         | 3（forget / input / output） | 2（update / reset） |
| Cell state   | 有 C_t                      | ❌ 没有              |
| Hidden state | h_t                        | h_t               |
| 参数量          | 多                          | 少                 |
| 计算复杂度        | 高                          | 低                 |

<center><img src="https://i-blog.csdnimg.cn/blog_migrate/63094448dc1aac09938d9333016d7856.png"></center>

### 1.更新门

$$z_{t}=\sigma\left(W_{z} \cdot\left[h_{t-1}, x_{t}\right]+b_{z}\right)$$

输入：[h_{t-1}, x_t]

输出：hidden_size

>这一部分决定当前时刻的hidden_state要保留多少过去的信息

## 2.重置门

$$r_{t}=\sigma\left(W_{r} \cdot\left[h_{t-1}, x_{t}\right]+b_{r}\right)$$

>这一部分决定在生成候选隐藏状态时，要不要保留以前的信息。

r_t ≈ 0
→ 忽略过去，像刚开始一个新句子

r_t ≈ 1
→ 强依赖过去上下文

## 3.候选隐藏状态

$$\tilde{h}_{t}=\tanh \left(W_{h} \cdot\left[r_{t} * h_{t-1}, x_{t}\right]+b_{h}\right)$$

>在是否使用历史信息的控制下，生成一个新的候选状态

## 4.GRU的最终hidden_state

$$h_{t}=\left(1-z_{t}\right) * \tilde{h}_{t}+z_{t} * h_{t-1}$$

当前状态 =

一部分来自“新信息”

一部分来自“旧记忆”

比例由更新门 z_t 决定

## 5.LSTM与GRU的参数量对比

假设：

输入维度 = d

隐状态维度 = h

LSTM 参数量（忽略 bias）

4 个线性变换（f, i, o, c）

每个：(d + h) × h

总计：4(d+h)h

GRU 参数量

3 个线性变换（z, r, h̃）

每个：(d + h) × h

总计：3(d+h)h

>GRU少了25%的参数

## 6.手写GRU Cell

In [1]:
import numpy as np

class GRUCell:
    def __init__(self, input_size, hidden_size):
        """
        input_size: x_t 的维度
        hidden_size: h_t 的维度
        """

        self.input_size = input_size
        self.hidden_size = hidden_size

        #z_t
        self.W_z = np.random.randn(hidden_size, input_size + hidden_size)
        self.b_z = np.zeros(hidden_size, 1)

        #r_t
        self.W_r = np.random.randn(hidden_size, input_size + hidden_size)
        self.b_r = np.zeros(hidden_size, 1)

        #h~_t
        self.W_h = np.random.randn(hidden_size, input_size + hidden_size)
        self.b_h = np.zeros(hidden_size, 1)

    def forward(self, x_t, h_prev):
        """
        x_t: 当前时间步输入，shape = (input_size, 1)
        h_prev: 上一时间步隐藏状态，shape = (hidden_size, 1)
        """

        #拼接输入和上一时刻的隐藏状态
        combined = np.concatenate((x_t, h_prev), axis=0)

        #calculate z_t
        z_t = self.sigmoid(np.dot(self.W_z, combined) + self.b_z)

        #calculate r_t
        r_t = self.sigmoid(np.dot(self.W_r, combined) + self.b_r)

        #calculate h~_t
        r_h_prev = r_t * h_prev
        combined_candidate = np.concatenate(r_h_prev, x_t)
        h_tilde = np.tanh(np.dot(self.W.h, r_h_prev) + self.b_h)

        #calculate h_t
        h_t = (1 - z_t) * h_tilde + z_t * h_prev

        return h_t
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-1))
